In [1]:
import numpy as np
from sklearn.model_selection import train_test_split

import torch
from torch import nn

from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import wandb

from training_utilities import train_loop, evaluation_loop, save_checkpoint, load_checkpoint

이번 실습시간에는 다양한 학습 전략과 hyperparameter tuning을 통해 CIFAR-10 테스트셋에서 높은 분류 성능을 얻는 것이 목표이다.

<mark>과제</mark> 다양한 조건에서 CIFAR-10 데이터셋 학습을 실험해보고 test 데이터셋에서 80% 이상의 accuracy를 달성하라.

* 제출물1 : <u>5개 이상의 학습 커브</u>를 포함하는 wandb 화면 캡처 (wandb 웹페이지의 본인 이름 포함하여 캡처)
* 제출물2 : 실험 결과에 대한 분석과 논의 (아래에 markdown으로 기입)

참고: 코드에 대한 pytest가 따로 없으므로 자유롭게 코드를 변경하여도 무방함.

단, <U>Transfer learning 혹은 Batch size는 변경은 수행하지 말것</U>

실험 조건 예시
- [Network architectures](https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate)
- input normalization
- [Weight initialization](https://pytorch.org/docs/stable/nn.init.html#torch.nn.init.kaiming_normal_)
- [Optimizers](https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate) (Adam, SGD with momentum, ... )
- Regularizations (weight decay, dropout, [Data augmentation](https://pytorch.org/vision/0.9/transforms.html), ensembles, ...)
- learning rate & [learning rate scheduler](https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate)

스스로 neural network를 구축할 경우 아래 사항들을 고려하라
- Filter size
- Number of filters
- Pooling vs Strided Convolution
- Activation functions

In [2]:
def get_model(model_name, num_classes, config):
    if model_name == "resnet50":
        model = models.resnet50()
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    else:
        raise Exception("Model not supported: {}".format(model_name))
    
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"Using model {model_name} with {total_params} parameters ({trainable_params} trainable)")

    return model


In [3]:
def load_cifar10_dataloaders(data_root_dir, device, batch_size, num_worker):
    validation_size = 0.2
    random_seed = 42

    normalize = transforms.Normalize(mean = (0.5, 0.5, 0.5), std = (0.5, 0.5, 0.5)) 
    
    train_transforms = transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])

    test_transforms = transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])

    train_dataset = datasets.CIFAR10(root=data_root_dir, train=True, download=True, transform=train_transforms)
    val_dataset = datasets.CIFAR10(root=data_root_dir, train=True, download=True, transform=test_transforms)
    test_dataset = datasets.CIFAR10(root=data_root_dir, train=False, download=True, transform=test_transforms)

    num_classes = len(train_dataset.classes)

    # Split train dataset into train and validataion dataset
    train_indices, val_indices = train_test_split(np.arange(len(train_dataset)), 
                                                  test_size=validation_size, random_state=random_seed)
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)

    # DataLoader
    kwargs = {}
    if device.startswith("cuda"):
        kwargs.update({
            'pin_memory': True,
        })

    train_dataloader = DataLoader(dataset = train_dataset, batch_size=batch_size, sampler=train_sampler,
                                  num_workers=num_worker, **kwargs)
    val_dataloader = DataLoader(dataset = val_dataset, batch_size=batch_size, sampler=valid_sampler,
                                num_workers=num_worker, **kwargs)
    test_dataloader = DataLoader(dataset = test_dataset, batch_size=batch_size, shuffle=False, 
                                 num_workers=num_worker, **kwargs)
    
    
    return train_dataloader, val_dataloader, test_dataloader, num_classes


In [4]:
def train_main(config):
    ## data and preprocessing settings
    data_root_dir = config['data_root_dir']
    num_worker = config.get('num_worker', 4)

    ## Hyper parameters
    batch_size = config['batch_size']
    learning_rate = config['learning_rate']
    start_epoch = config.get('start_epoch', 0)
    num_epochs = config['num_epochs']

    ## checkpoint setting
    checkpoint_save_interval = config.get('checkpoint_save_interval', 10)
    checkpoint_path = config.get('checkpoint_path', "checkpoints/checkpoint.pth")
    best_model_path = config.get('best_model_path', "checkpoints/best_model.pth")
    load_from_checkpoint = config.get('load_from_checkpoint', None)

    ## variables
    best_acc1 = 0

    wandb.init(
        project=config["wandb_project_name"],
        config=config
    )

    device = "cuda:0" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
    print(f"Using {device} device")

    train_dataloader, val_dataloader, test_dataloader, num_classes = load_cifar10_dataloaders(
        data_root_dir, device, batch_size = batch_size, num_worker = num_worker)
    
    model = get_model(model_name = config["model_name"], num_classes= num_classes, config = config).to(device)

    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1) 

    if load_from_checkpoint:
        load_checkpoint_path = (best_model_path if load_from_checkpoint == "best" else checkpoint_path)
        start_epoch, best_acc1 = load_checkpoint(load_checkpoint_path, model, optimizer, scheduler, device)

    if config.get('test_mode', False):
        # Only evaluate on the test dataset
        print("Running test evaluation...")
        test_acc = evaluation_loop(model, device, test_dataloader, criterion, phase = "test")
        print(f"Test Accuracy: {test_acc}")
        
    else:
        # Train and validate using train/val datasets
        for epoch in range(start_epoch, num_epochs):
            train_loop(model, device, train_dataloader, criterion, optimizer, epoch)
            val_acc1 = evaluation_loop(model, device, val_dataloader, criterion, epoch = epoch, phase = "validation")
            scheduler.step()

            if (epoch + 1) % checkpoint_save_interval == 0 or (epoch + 1) == num_epochs:
                is_best = val_acc1 > best_acc1
                best_acc1 = max(val_acc1, best_acc1)
                save_checkpoint(checkpoint_path, model, optimizer, scheduler, epoch, best_acc1, is_best, best_model_path)

    wandb.finish()


In [ ]:
config = {
    'data_root_dir': '/datasets',
    'batch_size': 64,
    'learning_rate': 1e-3,
    'num_epochs': 150,
    'model_name': 'resnet50',
    "dataset": "CIFAR10",
    'wandb_project_name': 'CIFAR10_hyperparameter_tuning',

    "checkpoint_save_interval" : 20,
    "checkpoint_path" : "checkpoints/checkpoint.pth",
    "best_model_path" : "checkpoints/best_model.pth",
    "load_from_checkpoint" : None,    # Options: "latest", "best", or None
}
train_main(config)


실험이 모두 끝나면 best model에 대해 test set성능을 평가한다. 

In [ ]:
config_testmode = {
    **config, 
    'test_mode': True, # True if evaluating only test set
    'load_from_checkpoint': 'best'
}

train_main(config_testmode)

<mark>제출물</mark>

1. 본인 이름이 나오도록 wandb 결과 화면을 캡처하여 `YOUR_PRIVATE_REPOSITORY_NAME/lab_05/wandb_results.png`에 저장한다. (5 points)
2. 결과를 table로 정리한 뒤 그 아래에 분석 및 논의를 작성 한다. (15 points)

-----

#### wandb 결과

<center><img src="./wandb_results.png" width="1000px"></img></center>

#### 5개 이상의 실험 결과

| 모델 | 실험 조건 | val_accuracy | 설명  |
|------|----------|--------------|------|
|      |          |              |      |
|      |          |              |      |
|      |          |              |      |
|      |          |              |      |
|      |          |              |      |

best model test_set accuracy: 

#### 분석 및 논의

-----
#### Lab을 마무리 짓기 전 저장된 checkpoint를 모두 지워 저장공간을 확보한다

In [1]:
import shutil, os
if os.path.exists('checkpoints/'):
    shutil.rmtree('checkpoints/')